In [ ]:
# %matplotlib tk
import matplotlib
matplotlib.use('Qt5Agg')  # 使用 Qt5 后端
import matplotlib.pyplot as plt
from matplotlib import font_manager
from concurrent.futures import ThreadPoolExecutor
import numpy as np

from einops import rearrange, repeat

from compute import *

# hype parameter
model_name = 'EdSr'
control_name = 'MD'
benchmark_name = 'BM'
""" Temp Press PotEng KinEng Enthalpy E_vdwl E_coul E_pair E_bond E_angle E_dihed E_long E_tail E_mol Ecouple Econserve TotEng Lx Ly Lz"""
energy_unit = "kcal $\\cdot$ mol$^{-1}$"
press_unit = "ATM"
temperature = "K"
distance_unit = "Angstrom"
time_unit = "fs"
time_scale = {
    "fs": 1,
    "ps": 1000,
    "ns": 1000000,
}
xaxis_time_unit = 'ps'
thermo_style_unit = {
    'temp'     : f"Kelvin ({temperature})",    'Temp'     : f'Kelvin ({temperature})',
    'press'    : f'ATMosphere ({press_unit})', 'Press'    : f'ATMosphere ({press_unit})',
    "pe"       : f'energy ({energy_unit})',    'PotEng'   : f'energy ({energy_unit})',
    "ke"       : f'energy ({energy_unit})',    'KinEng'   : f'energy ({energy_unit})',
    "enthalpy" : f'energy ({energy_unit})',    'Enthalpy' : f'energy ({energy_unit})',
    "evdwl"    : f'energy ({energy_unit})',    'E_vdwl'   : f'energy ({energy_unit})',
    "ecoul"    : f'energy ({energy_unit})',    'E_coul'   : f'energy ({energy_unit})',
    "epair"    : f'energy ({energy_unit})',    'E_pair'   : f'energy ({energy_unit})',
    "ebond"    : f'energy ({energy_unit})',    'E_bond'   : f'energy ({energy_unit})',
    "eangle"   : f'energy ({energy_unit})',    'E_angle'  : f'energy ({energy_unit})',
    "edihed"   : f'energy ({energy_unit})',    'E_dihed'  : f'energy ({energy_unit})',
    "eimp"     : f'energy ({energy_unit})',
    "elong"    : f'energy ({energy_unit})',    'E_long'   : f'energy ({energy_unit})',
    "etail"    : f'energy ({energy_unit})',    'E_tail'   : f'energy ({energy_unit})',
    "emol"     : f'energy ({energy_unit})',    'E_mol'    : f'energy ({energy_unit})',
    "ecouple"  : f'energy ({energy_unit})',    'Ecouple'  : f'energy ({energy_unit})',
    "econserve": f'energy ({energy_unit})',    'Econserve': f'energy ({energy_unit})',
    "etotal"   : f'energy ({energy_unit})',    'TotEng'   : f'energy ({energy_unit})',
    'lx'       : f'length ({distance_unit})',  'Lx'       : f'length ({distance_unit})',
    'ly'       : f'length ({distance_unit})',  'Ly'       : f'length ({distance_unit})',
    'lz'       : f'length ({distance_unit})',  'Lz'       : f'length ({distance_unit})',
}

title_mapping = {
     'temp'     : r'Temperature',                                                'Temp'     : r'Temperature',
     'press'    : r'Pressure',                                                   'Press'    : r'Pressure',
     "pe"       : r'Potential energy',                                           'PotEng'   : r'potential energy',
     "ke"       : r'Kinetic energy',                                             'KinEng'   : r'Kinetic energy',
     "enthalpy" : r'Total energy (pe + ke)',                                     'Enthalpy' : r'Total energy (pe + ke)',
     "evdwl"    : r'Van der Waals pairwise energy',                              'E_vdwl'   : r'Van der Waals pairwise energy',
     "ecoul"    : r'Coulombic pairwise energy',                                  'E_coul'   : r'Coulombic pairwise energy',
     "epair"    : r'Pairwise energy',                                            'E_pair'   : r'Pairwise energy',
     "ebond"    : r'Bond energy',                                                'E_bond'   : r'Bond energy',
     "eangle"   : r'Angle energy',                                               'E_angle'  : r'Angle energy',
     "edihed"   : r'Dihedral energy',                                            'E_dihed'  : r'Dihedral energy',
     "eimp"     : r'Improper energy',
     "elong"    : r'Long-range kspace energy',                                   'E_long'   : r'Long-range kspace energy',
     "etail"    : r'Van der Waals energy long-range tail correction',            'E_tail'   : r'Van der Waals energy long-range tail correction',
     "emol"     : r'Intramolecular energy',                                      'E_mol'    : r'Intramolecular energy',
     "ecouple"  : r'Cumulative energy change due to thermo/baro statting fixes', 'Ecouple'  : r'Cumulative energy change due to thermo/baro statting fixes',
     "econserve": r'Etotal + ecouple',                                           'Econserve': r'Etotal + ecouple',
     "etotal"   : r'Total energy',                                               'TotEng'   : r'Total energy',
     'lx'       : r'Length of x-axis',                                           'Lx'       : r'Length of x-axis',
     'ly'       : r'Length of y-axis',                                           'Ly'       : r'Length of y-axis',
     'lz'       : r'Length of z-axis',                                           'Lz'       : r'Length of z-axis',
     'Rg'       : r'Radius of gyration',                                         'RG'       : r'Radius of gyration',
     'rmsd'     : r'RMSD',                                                       'RMSD'     : r'RMSD',
     'msd'      : r'MSD',                                                        'MSD'      : r'MSD',
}



np.set_printoptions(threshold = np.inf)

fontsize = font_manager.FontProperties(size = 12)
tick_fontsize = font_manager.FontProperties(size = 10)
title_fontsize = font_manager.FontProperties(size = 11)
legned_fontsize = font_manager.FontProperties(size = 7)
item_fontsize = font_manager.FontProperties(size = 12)

tight_layout_arg = dict(
    top=0.906,
    bottom=0.106,
    left=0.05,
    right=0.996,
    hspace=0.267,
    wspace=0.257
)

row, col = 2, 3

fig = plt.figure(figsize=[10*col,20*row], dpi=256)

pic_idx = 0
subgraph_item = 0 + ord('a') - 1
item_pos = (-0.075, 1.1)

## $$\textit{ 20.0\ fs }$$

In [ ]:
# intv10
maxIter: int   = 500
intv   : int   = 20
basis  : float = 1.0
start  : int   = 1
end    : int   = 10000
group  : str   = "id 1 163"

benchmark_params = [
    f'data/benchmark_nve_basis{basis}_intv{intv}/',
    f'frames{start}_{end}.npz', 
    f'GlobalVariable.npz',
    f'rdf_pro2wa.dat',
]
control_params = [
    f'data/control_nve_basis{basis}_intv{intv}/',
    f'frames{start}_{end}.npz', 
    f'GlobalVariable.npz',
    f'rdf_pro2wa.dat',
]
taylor_params = [
    f'data/taylor_nve_basis{basis}_intv{intv}_iter500/',
    f'frames{start}_{end}.npz', 
    f'GlobalVariable.npz',
    f'rdf_pro2wa.dat',
]

# mass shape: (natoms,), id shape: (natoms,),  x shape: (ntrajs, natoms, 3), v shape: (ntrajs, natoms, 3)
with ThreadPoolExecutor(max_workers = 3) as executor:
    futures = [
        executor.submit(extraction, *benchmark_params[:-1], filter = None),
        executor.submit(extraction, *control_params[:-1], filter = None),
        executor.submit(extraction, *taylor_params[:-1] , filter = None),
    ]

benchmark_x, benchmark_v, benchmark_delta_t, benchmark_mass, benchmark_atype, benchmark_id, benchmark_boundary, benchmark_heads, benchmark_ppties, benchmark_init_state, benchmark_last_state = futures[0].result()
control_x, control_v, control_delta_t, control_mass, control_atype, control_id, control_boundary, control_heads, control_ppties, control_init_state, control_last_state = futures[1].result()
taylor_x, taylor_v, taylor_delta_t, taylor_mass, taylor_atype, taylor_id, taylor_boundary, taylor_heads, taylor_ppties, taylor_init_state, taylor_last_state = futures[2].result()

times = np.arange(end)*intv*basis * (time_scale[time_unit]/time_scale[xaxis_time_unit])

In [ ]:
# kinetic energy
ppty = 'ke'

ke_bidx, = np.where(benchmark_heads == ppty); bke = benchmark_ppties[:, ke_bidx]
ke_cidx, = np.where(control_heads == ppty); cke = control_ppties[:, ke_cidx]
ke_tidx, = np.where(taylor_heads == ppty); tke = taylor_ppties[:, ke_tidx]

ke_diff_bc = np.fabs(cke - bke)
ke_diff_bt = np.fabs(tke - bke)

# ! Picture idx
pic_idx += 1
subgraph_item += 1

skip = 200
if skip > 1:
    gke_diff_bc = np.pad(np.mean(ke_diff_bc.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = ke_diff_bc[0])
    gke_diff_bt = np.pad(np.mean(ke_diff_bt.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = ke_diff_bt[0])
    group_times = np.pad(times.reshape(-1, skip)[:, -1], pad_width = (1, 0), mode = 'constant', constant_values = 0)
    skip_bke = np.pad(np.mean(bke.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = bke[0])
    skip_cke = np.pad(np.mean(cke.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = cke[0])
    skip_tke = np.pad(np.mean(tke.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = tke[0])
else:
    gke_diff_bc = ke_diff_bc
    gke_diff_bt = ke_diff_bt
    group_times = times

ax = plt.subplot(row, col, pic_idx)
plt.text(*item_pos, chr(subgraph_item), transform = ax.transAxes, fontsize = item_fontsize.get_size(), fontweight = 'bold', va = 'top', ha = 'left')

# ! title of column 
plt.text(0.5, 1.15, f"{title_mapping[ppty]}\n($\\mathrm{{kcal \\cdot  mol^{{-1}}}}$)", transform = ax.transAxes, fontsize = title_fontsize.get_size(), fontweight = 'bold', va = 'center', ha = 'center')

plt.plot(group_times, skip_bke, label = f"{benchmark_name}", linewidth = 2)
plt.plot(group_times, skip_tke, label = f"{model_name}", linewidth = 1, alpha = 0.7)
plt.plot(group_times, skip_cke, label = f"{control_name}", linewidth = 1, alpha = 0.7)


plt.ticklabel_format(axis = 'y', style = 'scientific', scilimits = (0, 2), useMathText = True, useLocale = True)
plt.tick_params(axis = 'both', labelsize = tick_fontsize.get_size())
plt.xticks([0*round(basis * intv, 1), 2*round(basis * intv, 1), 4*round(basis * intv, 1), 6*round(basis * intv, 1), 8*round(basis * intv, 1), 10*round(basis * intv, 1)])
plt.ylim(bottom = 125)
plt.subplots_adjust(**tight_layout_arg)
plt.legend(fontsize = legned_fontsize.get_size(), loc = 'lower right', ncol = 1)

box1 = ax.get_position()
x0, y0, x1, y1 = box1.x0, box1.y0, box1.x1, box1.y1
aia = fig.add_axes([x0 + 0.07, y0 + 0.01, 0.1, 0.11])
aia.set_title('MAE', fontdict = {'fontsize': title_fontsize.get_size() - 3, 'verticalalignment': 'top', 'fontweight': 'bold'})
aia.axhline(y = np.mean(ke_diff_bt), color = 'orange', linestyle = '--', linewidth = 2, label = f'{model_name}')
aia.axhline(y = np.mean(ke_diff_bc), color = 'g', linestyle = '--', linewidth = 1, label = f'{control_name}')
aia.tick_params(axis = 'y', labelsize = tick_fontsize.get_size() - 4)
aia.set_xticks([])
aia.set_yticks([round(min(np.mean(ke_diff_bt), np.mean(ke_diff_bc)), 2), round(max(np.mean(ke_diff_bt), np.mean(ke_diff_bc)), 2)])
aia.set_ylim(bottom = 7.5, top = 8.2)
aia.legend(fontsize = legned_fontsize.get_size() - 3, loc = 'lower right', ncol = 2)

In [ ]:
# Pairwise energy
ppty = 'epair'

epair_bidx, = np.where(benchmark_heads == ppty); bepair = benchmark_ppties[:, epair_bidx]
epair_cidx, = np.where(control_heads == ppty); cepair = control_ppties[:, epair_cidx]
epair_tidx, = np.where(taylor_heads == ppty); tepair = taylor_ppties[:, epair_tidx]

epair_diff_bc = np.fabs(cepair - bepair)
epair_diff_bt = np.fabs(tepair - bepair)

# ! Picture idx
pic_idx += 1
subgraph_item += 1

skip = 200
if skip > 1:
    gepair_diff_bc = np.pad(np.mean(epair_diff_bc.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = epair_diff_bc[0])
    gepair_diff_bt = np.pad(np.mean(epair_diff_bt.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = epair_diff_bt[0])
    group_times = np.pad(times.reshape(-1, skip)[:, -1], pad_width = (1, 0), mode = 'constant', constant_values = 0)
    skip_bepair = np.pad(np.mean(bepair.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = bepair[0])
    skip_cepair = np.pad(np.mean(cepair.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = cepair[0])
    skip_tepair = np.pad(np.mean(tepair.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = tepair[0])
else:
    gepair_diff_bc = epair_diff_bc
    gepair_diff_bt = epair_diff_bt
    group_times = times

ax = plt.subplot(row, col, pic_idx)
plt.text(*item_pos, chr(subgraph_item), transform = ax.transAxes, fontsize = item_fontsize.get_size(), fontweight = 'bold', va = 'top', ha = 'left')

# ! title of column 
plt.text(0.5, 1.15, f"{title_mapping[ppty]}\n($\\mathrm{{kcal \\cdot  mol^{{-1}}}}$)", transform = ax.transAxes, fontsize = title_fontsize.get_size(), fontweight = 'bold', va = 'center', ha = 'center')

plt.plot(group_times, skip_bepair, label = f"{benchmark_name}", linewidth = 2)
plt.plot(group_times, skip_tepair, label = f"{model_name}", linewidth = 1, alpha = 0.7)
plt.plot(group_times, skip_cepair, label = f"{control_name}", linewidth = 1, alpha = 0.7)

plt.xticks([0*round(basis * intv, 1), 2*round(basis * intv, 1), 4*round(basis * intv, 1), 6*round(basis * intv, 1), 8*round(basis * intv, 1), 10*round(basis * intv, 1)])

plt.ticklabel_format(axis = 'y', style = 'scientific', scilimits = (0, 2), useMathText = True, useLocale = True)
plt.tick_params(axis = 'both', labelsize = tick_fontsize.get_size())
ax.invert_yaxis()
plt.ylim(bottom = -980.0)
plt.subplots_adjust(**tight_layout_arg)
plt.legend(fontsize = legned_fontsize.get_size(), loc = 'lower right', ncol = 1)

box2 = ax.get_position()
x0, y0, x1, y1 = box2.x0, box2.y0, box2.x1, box2.y1
aia = fig.add_axes([x0 + 0.07, y0 + 0.01, 0.1, 0.11])
aia.set_title('MAE', fontdict = {'fontsize': title_fontsize.get_size() - 3, 'verticalalignment': 'top', 'fontweight': 'bold'})
aia.axhline(y = np.mean(epair_diff_bt), color = 'orange', linestyle = '--', linewidth = 2, label = f'{model_name}')
aia.axhline(y = np.mean(epair_diff_bc), color = 'g', linestyle = '--', linewidth = 1, label = f'{control_name}')
aia.tick_params(axis = 'y', labelsize = tick_fontsize.get_size() - 4)
aia.set_xticks([])
aia.set_yticks([round(min(np.mean(epair_diff_bt), np.mean(epair_diff_bc)), 2), round(max(np.mean(epair_diff_bt), np.mean(epair_diff_bc)), 2)])
aia.set_ylim(bottom = 8)
aia.legend(fontsize = legned_fontsize.get_size() - 3, loc = 'lower right', ncol = 2)

In [ ]:
# Molecular energy
ppty = 'emol'

emol_bidx, = np.where(benchmark_heads == ppty); bemol = benchmark_ppties[:, emol_bidx]
emol_cidx, = np.where(control_heads == ppty); cemol = control_ppties[:, emol_cidx]
emol_tidx, = np.where(taylor_heads == ppty); temol = taylor_ppties[:, emol_tidx]

emol_diff_bc = np.fabs(cemol - bemol)
emol_diff_bt = np.fabs(temol - bemol)

# ! Picture idx
pic_idx += 1
subgraph_item += 1

skip = 200
if skip > 1:
    gemol_diff_bc = np.pad(np.mean(emol_diff_bc.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = emol_diff_bc[0])
    gemol_diff_bt = np.pad(np.mean(emol_diff_bt.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = emol_diff_bt[0])
    group_times = np.pad(times.reshape(-1, skip)[:, -1], pad_width = (1, 0), mode = 'constant', constant_values = 0)
    skip_bemol = np.pad(np.mean(bemol.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = bemol[0])
    skip_cemol = np.pad(np.mean(cemol.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = cemol[0])
    skip_temol = np.pad(np.mean(temol.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = temol[0])
else:
    gemol_diff_bc = emol_diff_bc
    gemol_diff_bt = emol_diff_bt
    group_times = times

ax = plt.subplot(row, col, pic_idx)
plt.text(*item_pos, chr(subgraph_item), transform = ax.transAxes, fontsize = item_fontsize.get_size(), fontweight = 'bold', va = 'top', ha = 'left')

# ! title of column 
plt.text(0.5, 1.15, f"{title_mapping[ppty]}\n($\\mathrm{{kcal \\cdot  mol^{{-1}}}}$)", transform = ax.transAxes, fontsize = title_fontsize.get_size(), fontweight = 'bold', va = 'center', ha = 'center')

plt.plot(group_times, skip_bemol, label = f"{benchmark_name}", linewidth = 2)
plt.plot(group_times, skip_temol, label = f"{model_name}", linewidth = 1, alpha = 0.7)
plt.plot(group_times, skip_cemol, label = f"{control_name}", linewidth = 1, alpha = 0.7)

plt.xticks([0*round(basis * intv, 1), 2*round(basis * intv, 1), 4*round(basis * intv, 1), 6*round(basis * intv, 1), 8*round(basis * intv, 1), 10*round(basis * intv, 1)])
plt.tick_params(axis = 'both', labelsize = tick_fontsize.get_size())
plt.ylim(bottom = 80.0)
plt.subplots_adjust(**tight_layout_arg)
plt.legend(fontsize = legned_fontsize.get_size(), loc = 'lower right', ncol = 1)

box3 = ax.get_position()
x0, y0, x1, y1 = box3.x0, box3.y0, box3.x1, box3.y1
aia = fig.add_axes([x0 + 0.07, y0 + 0.01, 0.1, 0.11])
aia.set_title('MAE', fontdict = {'fontsize': title_fontsize.get_size() - 3, 'verticalalignment': 'top', 'fontweight': 'bold'})
aia.axhline(y = np.mean(emol_diff_bt), color = 'orange', linestyle = '--', linewidth = 2, label = f'{model_name}')
aia.axhline(y = np.mean(emol_diff_bc), color = 'g', linestyle = '--', linewidth = 1, label = f'{control_name}')
aia.tick_params(axis = 'y', labelsize = tick_fontsize.get_size() - 4)
aia.set_xticks([])
aia.set_yticks([6.3, 6.7, 7.1])
aia.set_ylim(bottom = 6, top = 7.4)
aia.legend(fontsize = legned_fontsize.get_size() - 3, loc = 'lower right', ncol = 2)

## $$\textit{ 30.0\ fs }$$

In [ ]:
# intv10
maxIter: int   = 500
intv   : int   = 30
basis  : float = 1.0
start  : int   = 1
end    : int   = 10000
group  : str   = "id 1 163"

benchmark_params = [
    f'data/benchmark_nve_basis{basis}_intv{intv}/',
    f'frames{start}_{end}.npz', 
    f'GlobalVariable.npz',
    f'rdf_pro2wa.dat',
]
taylor_params = [
    f'data/taylor_nve_basis{basis}_intv{intv}_iter500/',
    f'frames{start}_{end}.npz', 
    f'GlobalVariable.npz',
    f'rdf_pro2wa.dat',
]

# mass shape: (natoms,), id shape: (natoms,),  x shape: (ntrajs, natoms, 3), v shape: (ntrajs, natoms, 3)
with ThreadPoolExecutor(max_workers = 3) as executor:
    futures = [
        executor.submit(extraction, *benchmark_params[:-1], filter = None),
        executor.submit(extraction, *taylor_params[:-1] , filter = None),
    ]

benchmark_x, benchmark_v, benchmark_delta_t, benchmark_mass, benchmark_atype, benchmark_id, benchmark_boundary, benchmark_heads, benchmark_ppties, benchmark_init_state, benchmark_last_state = futures[0].result()
taylor_x, taylor_v, taylor_delta_t, taylor_mass, taylor_atype, taylor_id, taylor_boundary, taylor_heads, taylor_ppties, taylor_init_state, taylor_last_state = futures[1].result()

times = np.arange(end)*intv*basis * (time_scale[time_unit]/time_scale[xaxis_time_unit])

In [ ]:
# kinetic energy
ppty = 'ke'

ke_bidx, = np.where(benchmark_heads == ppty); bke = benchmark_ppties[:, ke_bidx]
ke_tidx, = np.where(taylor_heads == ppty); tke = taylor_ppties[:, ke_tidx]

ke_diff_bt = np.fabs(tke - bke)

# ! Picture idx
pic_idx += 1
subgraph_item += 1

skip = 200
if skip > 1:
    gke_diff_bt = np.pad(np.mean(ke_diff_bt.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = ke_diff_bt[0])
    group_times = np.pad(times.reshape(-1, skip)[:, -1], pad_width = (1, 0), mode = 'constant', constant_values = 0)
    skip_bke = np.pad(np.mean(bke.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = bke[0])
    skip_tke = np.pad(np.mean(tke.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = tke[0])
else:
    gke_diff_bt = ke_diff_bt
    group_times = times

ax = plt.subplot(row, col, pic_idx)
plt.text(*item_pos, chr(subgraph_item), transform = ax.transAxes, fontsize = item_fontsize.get_size(), fontweight = 'bold', va = 'top', ha = 'left')

plt.plot(group_times, skip_bke, label = f"{benchmark_name}", linewidth = 2)
plt.plot(group_times, skip_tke, label = f"{model_name}", linewidth = 1, alpha = 0.7)


plt.ticklabel_format(axis = 'y', style = 'scientific', scilimits = (0, 2), useMathText = True, useLocale = True)
plt.xlabel(f't ($\\mathrm{{{xaxis_time_unit}}}$)', fontproperties = fontsize, fontweight = 'bold');
plt.tick_params(axis = 'both', labelsize = tick_fontsize.get_size())
plt.xticks([0*round(basis * intv, 1), 2*round(basis * intv, 1), 4*round(basis * intv, 1), 6*round(basis * intv, 1), 8*round(basis * intv, 1), 10*round(basis * intv, 1)])
plt.ylim(bottom = 95)
plt.subplots_adjust(**tight_layout_arg)
plt.legend(fontsize = legned_fontsize.get_size(), loc = 'lower right', ncol = 1)

box1 = ax.get_position()
x0, y0, x1, y1 = box1.x0, box1.y0, box1.x1, box1.y1
aia = fig.add_axes([x0 + 0.07, y0 + 0.01, 0.1, 0.11])
aia.set_title('MAE', fontdict = {'fontsize': title_fontsize.get_size() - 3, 'verticalalignment': 'top', 'fontweight': 'bold'})
aia.axhline(y = np.mean(ke_diff_bt), color = 'g', linestyle = '--', linewidth = 2, label = f'{model_name}')
aia.tick_params(axis = 'y', labelsize = tick_fontsize.get_size() - 4)
aia.set_xticks([])
aia.legend(fontsize = legned_fontsize.get_size() - 3, loc = 'lower right', ncol = 2)

In [ ]:
# Pairwise energy
ppty = 'epair'

epair_bidx, = np.where(benchmark_heads == ppty); bepair = benchmark_ppties[:, epair_bidx]
epair_tidx, = np.where(taylor_heads == ppty); tepair = taylor_ppties[:, epair_tidx]

epair_diff_bt = np.fabs(tepair - bepair)

# ! Picture idx
pic_idx += 1
subgraph_item += 1

skip = 200
if skip > 1:
    gepair_diff_bt = np.pad(np.mean(epair_diff_bt.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = epair_diff_bt[0])
    group_times = np.pad(times.reshape(-1, skip)[:, -1], pad_width = (1, 0), mode = 'constant', constant_values = 0)
    skip_bepair = np.pad(np.mean(bepair.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = bepair[0])
    skip_tepair = np.pad(np.mean(tepair.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = tepair[0])
else:
    gepair_diff_bt = epair_diff_bt
    group_times = times

ax = plt.subplot(row, col, pic_idx)
plt.text(*item_pos, chr(subgraph_item), transform = ax.transAxes, fontsize = item_fontsize.get_size(), fontweight = 'bold', va = 'top', ha = 'left')

plt.plot(group_times, skip_bepair, label = f"{benchmark_name}", linewidth = 2)
plt.plot(group_times, skip_tepair, label = f"{model_name}", linewidth = 1, alpha = 0.7)
plt.xticks([0*round(basis * intv, 1), 2*round(basis * intv, 1), 4*round(basis * intv, 1), 6*round(basis * intv, 1), 8*round(basis * intv, 1), 10*round(basis * intv, 1)])
plt.xlabel(f't ($\\mathrm{{{xaxis_time_unit}}}$)', fontproperties = fontsize, fontweight = 'bold');
plt.ticklabel_format(axis = 'y', style = 'scientific', scilimits = (0, 2), useMathText = True, useLocale = True)
plt.tick_params(axis = 'both', labelsize = tick_fontsize.get_size())
ax.invert_yaxis()
plt.ylim(bottom = -970.0)
plt.subplots_adjust(**tight_layout_arg)
plt.legend(fontsize = legned_fontsize.get_size(), loc = 'lower right', ncol = 1)

box2 = ax.get_position()
x0, y0, x1, y1 = box2.x0, box2.y0, box2.x1, box2.y1
aia = fig.add_axes([x0 + 0.07, y0 + 0.01, 0.1, 0.11])
aia.set_title('MAE', fontdict = {'fontsize': title_fontsize.get_size() - 3, 'verticalalignment': 'top', 'fontweight': 'bold'})
aia.axhline(y = np.mean(epair_diff_bt), color = 'g', linestyle = '--', linewidth = 2, label = f'{model_name}')
aia.tick_params(axis = 'y', labelsize = tick_fontsize.get_size() - 4)
aia.set_xticks([])
aia.legend(fontsize = legned_fontsize.get_size() - 3, loc = 'lower right', ncol = 2)

In [ ]:
# Molecular energy
ppty = 'emol'

emol_bidx, = np.where(benchmark_heads == ppty); bemol = benchmark_ppties[:, emol_bidx]
emol_tidx, = np.where(taylor_heads == ppty); temol = taylor_ppties[:, emol_tidx]

emol_diff_bt = np.fabs(temol - bemol)

# ! Picture idx
pic_idx += 1
subgraph_item += 1

skip = 200
if skip > 1:
    gemol_diff_bt = np.pad(np.mean(emol_diff_bt.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = emol_diff_bt[0])
    group_times = np.pad(times.reshape(-1, skip)[:, -1], pad_width = (1, 0), mode = 'constant', constant_values = 0)
    skip_bemol = np.pad(np.mean(bemol.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = bemol[0])
    skip_temol = np.pad(np.mean(temol.reshape(-1, skip), axis = -1), pad_width = (1, 0), mode = 'constant', constant_values = temol[0])
else:
    gemol_diff_bt = emol_diff_bt
    group_times = times

ax = plt.subplot(row, col, pic_idx)
plt.text(*item_pos, chr(subgraph_item), transform = ax.transAxes, fontsize = item_fontsize.get_size(), fontweight = 'bold', va = 'top', ha = 'left')

plt.plot(group_times, skip_bemol, label = f"{benchmark_name}", linewidth = 2)
plt.plot(group_times, skip_temol, label = f"{model_name}", linewidth = 1, alpha = 0.7)

plt.xticks([0*round(basis * intv, 1), 2*round(basis * intv, 1), 4*round(basis * intv, 1), 6*round(basis * intv, 1), 8*round(basis * intv, 1), 10*round(basis * intv, 1)])
plt.tick_params(axis = 'both', labelsize = tick_fontsize.get_size())
plt.xlabel(f't ($\\mathrm{{{xaxis_time_unit}}}$)', fontproperties = fontsize, fontweight = 'bold');
plt.ylim(bottom = 70.0)
plt.subplots_adjust(**tight_layout_arg)
plt.legend(fontsize = legned_fontsize.get_size(), loc = 'lower right', ncol = 1)

box3 = ax.get_position()
x0, y0, x1, y1 = box3.x0, box3.y0, box3.x1, box3.y1
aia = fig.add_axes([x0 + 0.07, y0 + 0.01, 0.1, 0.11])
aia.set_title('MAE', fontdict = {'fontsize': title_fontsize.get_size() - 3, 'verticalalignment': 'top', 'fontweight': 'bold'})
aia.axhline(y = np.mean(emol_diff_bt), color = 'g', linestyle = '--', linewidth = 2, label = f'{model_name}')
aia.tick_params(axis = 'y', labelsize = tick_fontsize.get_size() - 4)
aia.set_xticks([])
# aia.set_ylim(bottom = 10.5)
aia.legend(fontsize = legned_fontsize.get_size() - 3, loc = 'lower right', ncol = 2)

In [ ]:
plt.subplots_adjust(**tight_layout_arg)
plt.show()